In [ ]:
import tensorflow as tf 
import numpy as np
from keras import backend as K
from keras.models import load_model, Model
from PIL import Image, ImageDraw, ImageFont
import scipy.io
from matplotlib.pyplot import imshow
import os
import colorsys
import random
import imageio

In [ ]:
def filter_boxes(box_conf, boxes, box_class_probs, threshold):
  
  box_scores = box_conf*box_class_probs 
  box_class = K.argmax(box_scores, axis = -1) 
  box_class_score = K.max(box_scores, axis = -1) 
  
  mask = (box_class_score >= threshold) 
  print(mask)
  scores = tf.boolean_mask(box_class_score, mask, axis = None, name='scores') 
  boxes = tf.boolean_mask(boxes, mask, axis=None, name='boxes') 
  classes = tf.boolean_mask(box_class, mask, axis= None, name='classes') 

  return scores, boxes, classes

In [ ]:
def non_max_suppression(boxes,scores,classes,max_boxes = 10, iou_threshold = 0.5):
  max_boxes_tensor = K.variable(max_boxes, dtype='int32')
  K.get_session().run(tf.variables_initializer([max_boxes_tensor]))

  nms_indices = tf.image.non_max_suppression(boxes,scores,max_boxes,iou_threshold,name='nms_indices')

  scores = K.gather(scores, nms_indices)
  boxes = K.gather(boxes, nms_indices)
  classes = K.gather(classes,nms_indices)

  return scores, boxes, classes

In [ ]:
def boxes_to_corners(box_xy,box_wh):
    box_mins = box_xy - (box_wh / 2.)
    box_maxes = box_xy + (box_wh / 2.)

    return K.concatenate([
        box_mins[..., 1:2],  # y_min
        box_mins[..., 0:1],  # x_min
        box_maxes[..., 1:2],  # y_max
        box_maxes[..., 0:1]  # x_max
    ])


In [ ]:
def scale_boxes(boxes,img_shape):
    height = img_shape[0]
    width = img_shape[1]
    img_dims = K.stack([height,width,height,width])
    img_dims = K.reshape(img_dims, [1,4])

    return boxes*img_dims

In [ ]:
def yolo_inference(yolo_outputs, img_shape = (720., 1280.), max_boxes=10, score_threshold=.6, iou_threshold=.5):

    box_conf, box_xy, box_wh, box_class_probs = yolo_outputs
    boxes = boxes_to_corners(box_xy, box_wh)
    scores, boxes, classes = filter_boxes(box_conf, boxes, box_class_probs, score_threshold)
    boxes = scale_boxes(boxes, img_shape)
    scores, boxes, classes = non_max_suppression(boxes,scores,classes,max_boxes = 10, iou_threshold = 0.5)

    return scores, boxes, classes

In [ ]:
def read_anchors(path):
    with open(path) as f:
        anchors = f.readline()
        anchors = [float(x) for x in anchors.split(",")]
        anchors = np.array(anchors).reshape(-1,2)
    return anchors

def read_classes(path):
    with open(path) as f:
        classes = f.readlines()
    classes = [c.strip() for c in classes]
    return classes

In [ ]:
def preprocess_img(path, model_img_size):
    img = Image.open(path)
    resized_img = img.resize(tuple(reversed(model_img_size)), Image.BICUBIC)
    img_data = np.array(resized_img, dtype='float32')
    img_data /= 255.
    imag_data = np.expand_dims(img_data,0)
    return img, img_data

In [ ]:
def draw_boxes(img, out_scores, out_boxes, out_classes, class_names, colours):
    font = ImageFont.truetype(font='font.otf', size=np.floor(3e-2*img.size[1]+0.5).astype('int32'))
    thickness = (img.size[0] + img.size[1])//300

    for i,c in reversed(list(enumerate(out_classes))):
        predicted_class = class_names[c]
        box = out_boxes[i]
        score = out_scores[i]

        label ='{} {:.2f}'.format(predicted_class,score)

        draw = ImageDraw.Draw(img)
        label_size = draw.textsize(label, font)

        top, left, bottom, right = box
        top = max(0, np.floor(top + 0.5).astype('int32'))
        left = max(0, np.floor(left + 0.5).astype('int32'))
        bottom = min(img.size[1], np.floor(bottom + 0.5).astype('int32'))
        right = min(img.size[0], np.floor(right + 0.5).astype('int32'))
        print(label, (left, top), (right, bottom))

        if top - label_size[1] >= 0:
            text_origin = np.array([left, top - label_size[1]])
        else:
            text_origin = np.array([left, top + 1])

        for i in range(thickness):
            draw.rectangle([left + i, top + i, right - i, bottom - i], outline=colours[c])
        draw.rectangle([tuple(text_origin), tuple(text_origin + label_size)], fill=colours[c])
        draw.text(text_origin, label, fill=(0, 0, 0), font=font)
        del draw
        
def gen_colours(class_names):
    hsv_tuples = [(x / len(class_names), 1., 1.) for x in range(len(class_names))]
    colours = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
    colours = list(map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)), colours))
    random.seed(10101)  
    random.shuffle(colours)  
    random.seed(None)  
    return colours

In [ ]:
def yolo_opgen(feats, anchors, num_classes):

    num_anchors = len(anchors)
    anchors_tensor = K.reshape(K.variable(anchors), [1, 1, 1, num_anchors, 2])

    conv_dims = K.shape(feats)[1:3] 
    conv_height_index = K.arange(0, stop=conv_dims[0])
    conv_width_index = K.arange(0, stop=conv_dims[1])
    conv_height_index = K.tile(conv_height_index, [conv_dims[1]])

    conv_width_index = K.tile(K.expand_dims(conv_width_index, 0), [conv_dims[0], 1])
    conv_width_index = K.flatten(K.transpose(conv_width_index))
    conv_index = K.transpose(K.stack([conv_height_index, conv_width_index]))
    conv_index = K.reshape(conv_index, [1, conv_dims[0], conv_dims[1], 1, 2])
    conv_index = K.cast(conv_index, K.dtype(feats))
    
    feats = K.reshape(feats, [-1, conv_dims[0], conv_dims[1], num_anchors, num_classes + 5])
    conv_dims = K.cast(K.reshape(conv_dims, [1, 1, 1, 1, 2]), K.dtype(feats))

    box_conf = K.sigmoid(feats[..., 4:5])
    box_xy = K.sigmoid(feats[..., :2])
    box_wh = K.exp(feats[..., 2:4])
    box_class_probs = K.softmax(feats[..., 5:])

    box_xy = (box_xy + conv_index) / conv_dims
    box_wh = box_wh * anchors_tensor / conv_dims

    return box_conf, box_xy, box_wh, box_class_probs

In [ ]:
anchors = read_anchors("anchors.txt")
class_names = read_classes("classes80.txt")
yolo_model = load_model("yolo.h5")
img_size = (720.,1080.)
yolo_model.summary()

In [ ]:
sesh = K.get_session()
yolo_outputs = yolo_opgen(yolo_model.output, anchors, len(class_names))
scores, boxes, classes = yolo_inference(yolo_outputs)

In [ ]:
def predict(sesh, img_file):
    img, img_data = preprocess_img("images/" + img_file, model_img_size = (608,608))
    out_scores, out_boxes, out_classes = sesh.run(fetches = [scores,boxes,classes], feed_dict={yolo_model.input: img_data, K.learning_phase() : 0})
    print('Found {} boxes for {}'.format(len(out_boxes), img_file))
    colours = gen_colours(class_names)
    draw_boxes(img, out_scores, out_boxes, out_classes, class_names, colours)
    img.save(os.path.join("out/", img_file), quality=90)
    output_img = imageio.imread(os.path.join("out/", img_file))
    imshow(output_img)
    return out_scores, out_boxes, out_classes

In [ ]:
out_scores, out_boxes, out_classes = predict(sesh, "test.jpeg")